<a href="https://colab.research.google.com/github/AkibCoding/KD_ROBERTA/blob/main/Retry%26evaluateKDroberta2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import torch
from torch.nn import CrossEntropyLoss, CosineEmbeddingLoss
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from google.colab import drive
from transformers import RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

# Setting up device for GPU usage
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Tokenize the datasets with padding and truncation
def tokenize_batch(batch):
    return tokenizer(batch['sentence'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')

encoded_dataset = dataset.map(tokenize_batch, batched=True)

# Load the pre-trained RoBERTa-large model (teacher)
teacher_model = RobertaForSequenceClassification.from_pretrained("roberta-large")
teacher_model = teacher_model.to(device)
teacher_model.eval()  # Set the teacher model to evaluation mode

# Create a half-sized RoBERTa-base model (student)
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

student_model = create_half_size_roberta_base()
student_model = student_model.to(device)

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encoded_dataset, mode='train'):
        self.input_ids = torch.tensor(encoded_dataset[mode]['input_ids'])
        self.attention_mask = torch.tensor(encoded_dataset[mode]['attention_mask'])
        self.labels = torch.tensor(encoded_dataset[mode]['label'])  # Change 'labels' to 'label'

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]  # Change 'labels' to 'label'
        }

train_dataset = CustomDataset(encoded_dataset)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Distillation loss function
def distillation_loss(teacher_logits, student_logits, labels, temperature=1.0, alpha=0.5):
    kl_loss = torch.nn.KLDivLoss(reduction="batchmean")(torch.nn.functional.log_softmax(student_logits / temperature, dim=-1),
                            torch.nn.functional.softmax(teacher_logits / temperature, dim=-1))
    ce_loss = CrossEntropyLoss()(student_logits, labels)
    return alpha * kl_loss + (1. - alpha) * ce_loss

# Mount Google Drive
drive.mount('/content/drive')

# Define Saving and Loading Functions for the model
SAVE_DIR = "/content/drive/My Drive/Colab Notebooks/"
SAVE_PATH = os.path.join(SAVE_DIR, "student_model_state2.pth")

def save_model_to_drive(model, save_path=SAVE_PATH):
    torch.save(model.state_dict(), save_path)

def load_model_from_drive(model, save_path=SAVE_PATH):
    if os.path.exists(save_path):
        model.load_state_dict(torch.load(save_path))
        print("Model loaded from", save_path)
    else:
        print("No previous model state found!")

# Training loop with backup functionality
def train_student_custom_loop(teacher_model, student_model, train_dataloader, temperature=2.0):
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)

    epochs = 3
    for epoch in range(epochs):
        for i, batch in tqdm(enumerate(train_dataloader)):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            student_outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
            student_logits = student_outputs.logits

            with torch.no_grad():
                teacher_outputs = teacher_model(input_ids=input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits

            loss = distillation_loss(teacher_logits, student_logits, labels, temperature)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Save the student model after every 100 iterations
            if i % 100 == 0:
                save_model_to_drive(student_model)
                print(f"Epoch {epoch}, Iteration {i}, Loss: {loss.item()}")

# Load back the student model's state if available
load_model_from_drive(student_model)

# Train the student model
train_student_custom_loop(teacher_model, student_model, train_dataloader)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No previous model state found!


2it [00:04,  1.68s/it]

Epoch 0, Iteration 0, Loss: 0.3527374565601349


102it [00:29,  2.78it/s]

...

2702it [11:09,  2.92it/s]

Epoch 2, Iteration 2700, Loss: 0.12584859132766724


2802it [11:34,  3.04it/s]

Epoch 2, Iteration 2800, Loss: 0.17269471287727356


2885it [11:54,  4.13it/s]

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, Dataset
import os

# Setting up device for GPU usage
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Tokenize the datasets with padding and truncation
def tokenize_batch(batch):
    return tokenizer(batch['sentence'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')

encoded_dataset = dataset.map(tokenize_batch, batched=True)
# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encoded_dataset, mode='train'):
        self.input_ids = torch.tensor(encoded_dataset[mode]['input_ids'])
        self.attention_mask = torch.tensor(encoded_dataset[mode]['attention_mask'])
        self.labels = torch.tensor(encoded_dataset[mode]['label'])  # Change 'labels' to 'label'

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]  # Change 'labels' to 'label'
        }

train_dataset = CustomDataset(encoded_dataset)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# Define the student model creation function
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

student_model = create_half_size_roberta_base()
student_model = student_model.to(device)

# Load the student model from Google Drive
SAVE_DIR = "/content/drive/My Drive/Colab Notebooks/student_model_state2.pth"
SAVE_PATH = os.path.join(SAVE_DIR, "student_model_state3.pth")

def load_model_from_drive(model, save_path=SAVE_PATH):
    if os.path.exists(save_path):
        model.load_state_dict(torch.load(save_path))
        print("Model loaded from", save_path)
    else:
        print("No previous model state found!")

load_model_from_drive(student_model)

# Evaluation
student_model.eval()
correct = 0
total = 0

val_dataloader = DataLoader(CustomDataset(encoded_dataset, mode='validation'), batch_size=8)

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of the student model on the validation set: {accuracy:.2f}%")


Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

No previous model state found!
Accuracy of the student model on the validation set: 49.08%


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
from torch.utils.data import DataLoader

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create half-sized Roberta
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

# Load the student model
student_model = create_half_size_roberta_base()
student_model.to(device)

# Load weights from saved model
saved_model_path = "/content/drive/MyDrive/Colab Notebooks/student_model_state2.pth"
student_model.load_state_dict(torch.load(saved_model_path, map_location=device))

# Load and preprocess the SST-2 dataset
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
dataset = load_dataset("glue", "sst2")

# Encoding the dataset
def encode_dataset(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(encode_dataset, batched=True)

# Collation function
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'label': torch.stack([torch.tensor(item['label']) for item in batch])
    }

# DataLoader
val_dataloader = DataLoader(encoded_dataset["validation"], shuffle=False, batch_size=32, collate_fn=collate_fn)

# Evaluate the model
student_model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = student_model(inputs, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f"Accuracy of the loaded student model on the validation set: {accuracy:.2f}%")


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Accuracy of the loaded student model on the validation set: 80.39%


model performance on train set. seems the model is overfitted

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
from torch.utils.data import DataLoader

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create half-sized Roberta
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")  # Use the same architecture as the knowledge distillation
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

# Load the student model
student_model = create_half_size_roberta_base()
student_model.to(device)

# Load weights from saved model
saved_model_path = "/content/drive/MyDrive/Colab Notebooks/student_model_state2.pth"
student_model.load_state_dict(torch.load(saved_model_path, map_location=device))

# Load and preprocess the SST-2 dataset
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")  # Use the same tokenizer as the knowledge distillation

dataset = load_dataset("glue", "sst2")

# Encoding the dataset
def encode_dataset(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(encode_dataset, batched=True)

# Collation function
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'label': torch.stack([torch.tensor(item['label']) for item in batch])
    }

# DataLoader
val_dataloader = DataLoader(encoded_dataset["train"], shuffle=False, batch_size=32, collate_fn=collate_fn)

# Evaluate the model
student_model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = student_model(inputs, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f"Accuracy of the loaded student model on the validation set: {accuracy:.2f}%")


Accuracy of the loaded student model on the validation set: 92.88%


performance of same configuration but no KD

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
from torch.utils.data import DataLoader

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the RoBERTa Base tokenizer and configuration
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
config = RobertaConfig.from_pretrained("roberta-base")
config.hidden_size //= 2
config.num_attention_heads //= 2
config.intermediate_size //= 2

# Load the RoBERTa Base model based on the configuration
student_model = RobertaForSequenceClassification(config)
student_model.to(device)

# Load and preprocess the SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Encoding the validation dataset
def encode_dataset(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset["validation"].map(encode_dataset, batched=True)

# Collation function
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'label': torch.stack([torch.tensor(item['label']) for item in batch])
    }

# DataLoader
val_dataloader = DataLoader(encoded_dataset, shuffle=False, batch_size=32, collate_fn=collate_fn)

# Evaluate the model on the validation set
student_model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = student_model(inputs, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f"Accuracy of the RoBERTa Base model on the validation set: {accuracy:.2f}%")


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Accuracy of the RoBERTa Base model on the validation set: 50.46%


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig

def print_total_parameters_in_millions(model, model_name):
    print(f"Total Parameters for {model_name}:")
    total_params = sum(p.numel() for p in model.parameters())
    total_params_millions = total_params / 1_000_000  # Convert to millions
    print(f"{total_params_millions:.2f}M\n")  # Display 2 decimal points
    print("-" * 50 + "\n")

# Example usage:

# Initialize the teacher model
teacher_model = RobertaForSequenceClassification.from_pretrained("roberta-large")

# Initialize the student model
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

student_model = create_half_size_roberta_base()

print_total_parameters_in_millions(teacher_model, "Teacher Model")
print_total_parameters_in_millions(student_model, "Student Model")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters for Teacher Model:
355.36M

--------------------------------------------------

Total Parameters for Student Model:
40.94M

--------------------------------------------------



In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset

# Load the model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load dataset
dataset = load_dataset('glue', 'sst2', split='train')

# Shuffle the dataset to ensure randomness
dataset = dataset.shuffle(seed=42)

# Tokenize and prepare DataLoader
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

# Fine-tuning hyperparameters
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(data_loader)*10)  # Adjust warm-up steps

model = model.to(device)

# Fine-tune the model
num_epochs = 10  # Adjust the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for input_ids, attention_mask, label in data_loader:
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(data_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}')

# Evaluate the fine-tuned model
model = model.eval()
correct = 0
total = 0

dataset = load_dataset('glue', 'sst2', split='validation')
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

with torch.no_grad():
    for input_ids, attention_mask, label in data_loader:
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy after fine-tuning: {accuracy:.2f}%')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
# Evaluate the fine-tuned model
model = model.eval()
correct = 0
total = 0

dataset = load_dataset('glue', 'sst2', split='validation')
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

with torch.no_grad():
    for input_ids, attention_mask, label in data_loader:
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy after fine-tuning: {accuracy:.2f}%')

Accuracy after fine-tuning: 92.66%


Fine tuning part 2 to save the fine tune teacher model

In [ ]:
import os
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset

save_dir = '/content/drive/MyDrive/saved_models'
os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists
# Load the model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load dataset
dataset = load_dataset('glue', 'sst2', split='train')

# Shuffle the dataset to ensure randomness
dataset = dataset.shuffle(seed=42)

# Tokenize and prepare DataLoader
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

# Fine-tuning hyperparameters
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(data_loader)*10)  # Adjust warm-up steps

model = model.to(device)
# Fine-tune the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for input_ids, attention_mask, label in data_loader:
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

        _, predicted = torch.max(outputs.logits, 1)
        total_train += label.size(0)
        correct_train += (predicted == label).sum().item()

    average_loss = total_loss / len(data_loader)
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Optionally, add a validation phase to check model's performance on a validation set

    # Save the model after each epoch
    save_path = os.path.join(save_dir, f'teacher_epoch{epoch + 1}.pth')
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

# ... [Your evaluation code] ...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Average Loss: 0.6937, Training Accuracy: 54.49%
Model saved to /content/drive/MyDrive/saved_models/teacher_epoch1.pth


In [ ]:
import os
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from google.colab import drive


save_dir = '/content/drive/MyDrive/saved_models'
os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to the uploaded model file on Google Drive
uploaded_model_path = '/content/drive/MyDrive/saved_models/teacher_epoch1.pth'  # Update with your actual path

# Load the model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained weights from your uploaded checkpoint
if os.path.exists(uploaded_model_path):
    model.load_state_dict(torch.load(uploaded_model_path, map_location=device))
    print("Model loaded from", uploaded_model_path)
else:
    print("No pre-trained model checkpoint found at", uploaded_model_path)

# Load dataset
dataset = load_dataset('glue', 'sst2', split='train')

# Shuffle the dataset to ensure randomness
dataset = dataset.shuffle(seed=42)

# Tokenize and prepare DataLoader
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

# Fine-tuning hyperparameters
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(data_loader)*10)  # Adjust warm-up steps

model = model.to(device)
# Fine-tune the model
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for input_ids, attention_mask, label in data_loader:
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

        _, predicted = torch.max(outputs.logits, 1)
        total_train += label.size(0)
        correct_train += (predicted == label).sum().item()

    average_loss = total_loss / len(data_loader)
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch + 2}/{num_epochs}, Average Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Optionally, add a validation phase to check the model's performance on a validation set

    # Save the model after each epoch
    save_path = os.path.join(save_dir, f'teacher_epoch{epoch + 2}.pth')
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

# ... [Your evaluation code] ...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from /content/drive/MyDrive/saved_models/teacher_epoch1.pth
Epoch 2/4, Average Loss: 0.6940, Training Accuracy: 54.26%
Model saved to /content/drive/MyDrive/saved_models/teacher_epoch2.pth
Epoch 3/4, Average Loss: 0.6927, Training Accuracy: 54.43%
Model saved to /content/drive/MyDrive/saved_models/teacher_epoch3.pth


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset

# Load the model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Load the trained model weights
model_path = '/content/drive/MyDrive/saved_models/teacher_epoch3.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.eval()  # Set the model to evaluation mode


# Check if the model file exists
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.eval()  # Set the model to evaluation mode
    print(f'Model loaded from {model_path}. Proceeding to evaluation...')
else:
    print(f'No model found at {model_path}. Cannot proceed to evaluation.')

# Load dataset
dataset = load_dataset('glue', 'sst2', split='validation')

# Tokenize and prepare DataLoader
inputs = tokenizer([x['sentence'] for x in dataset], padding=True, return_tensors='pt', truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = torch.tensor(dataset['label']).to(device)
data_loader = DataLoader(TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels), batch_size=8)

# Evaluate the fine-tuned model
correct = 0
total = 0

with torch.no_grad():
    for input_ids, attention_mask, label in data_loader:
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy after fine-tuning: {accuracy:.2f}%')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from /content/drive/MyDrive/saved_models/teacher_epoch3.pth. Proceeding to evaluation...


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Accuracy after fine-tuning: 50.92%


Checking Teacher and Student Model parameters and Configurations

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig

def print_total_parameters_in_millions(model, model_name):
    print(f"Total Parameters for {model_name}:")
    total_params = sum(p.numel() for p in model.parameters())
    total_params_millions = total_params / 1_000_000  # Convert to millions
    print(f"{total_params_millions:.2f}M\n")  # Display 2 decimal points
    print("-" * 50 + "\n")

def print_model_config(model, model_name):
    print(f"Configuration for {model_name}:")
    print(model.config)  # Print the config of the model
    print("-" * 50 + "\n")

# Example usage:

# Initialize the teacher model
teacher_model = RobertaForSequenceClassification.from_pretrained("roberta-large")

# Initialize the student model
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

student_model = create_half_size_roberta_base()

print_total_parameters_in_millions(teacher_model, "Teacher Model")
print_total_parameters_in_millions(student_model, "Student Model")

# Printing the config of teacher and student model
print_model_config(teacher_model, "Teacher Model")
print_model_config(student_model, "Student Model")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters for Teacher Model:
355.36M

--------------------------------------------------

Total Parameters for Student Model:
40.94M

--------------------------------------------------

Configuration for Teacher Model:
RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

--------------------------------------------------

Configuration for Student Model:
RobertaConfig {
  "architect

Glue benchmarking on student model

In [ ]:
single_batch = next(iter(eval_dataloader))
print(single_batch['input_ids'])


[tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([  405,   879, 37984,   627,   405, 24648,   102,   368]), tensor([  128,  4825,   201,  3501,   128, 31923,  2128,   608]), tensor([   29,  3796,     7,  2156,    29,    19, 35138,    94]), tensor([   10,  7790,  1034, 14690,  2635, 12073,   822,    76]), tensor([18452, 23530,    14,  2156,   480,     8,   479,   128]), tensor([   8,    8,  295,  930,  182,   10, 1437,   29]), tensor([  747,  7764, 13887,  2156,  2156,   367,     2,  2556]), tensor([ 7920,  1437,    16, 18535,   182, 33639,     1,    19]), tensor([ 3251,     2, 10137, 34226,  2635, 16170,     1,   110]), tensor([  479,     1,     7,     8,   479, 12325,     1,  1931]), tensor([ 1437,     1, 14976,  2369,  1437,  2156,     1,    12]), tensor([    2,     1,    10,    32,     2,     5,     1, 12295]), tensor([  1,   1, 538,  70,   1, 822,   1, 479]), tensor([    1,     1,   756, 29932,     1,    16,     1,  1437]), tensor([  1,   1,  25, 576,   1,  10,   1,   2]), tensor([    1, 

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset, load_metric
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create half-sized Roberta
def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

# Load datasets
tasks = ["sst2", "mrpc", "cola"]
glue_data = {task: load_dataset("glue", task) for task in tasks}

# Load tokenizer and create student model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = create_half_size_roberta_base()
state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/student_model_state2.pth")
model.load_state_dict(state_dict)
model.to(device)
model.eval()

# Preprocess the datasets
def tokenize_function(examples, task):
    if task == "sst2" or task == "cola":
        text = examples['sentence']
    elif task == "mrpc":
        text = [s1 + " " + s2 for s1, s2 in zip(examples['sentence1'], examples['sentence2'])]
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

tokenized_datasets = {task: glue_data[task].map(lambda x: tokenize_function(x, task), batched=True) for task in tasks}

# Evaluation
metrics = {
    "sst2": load_metric("accuracy"),
    "mrpc": load_metric("f1"),
    "cola": load_metric("matthews_correlation")
}

results = {}

with torch.no_grad():
    for task in tasks:
        eval_dataloader = torch.utils.data.DataLoader(tokenized_datasets[task]["validation"], batch_size=8)

        all_predictions = []
        all_labels = []

        for batch in eval_dataloader:
            try:
                # Print type and content for debugging
                print(f"Type of batch['input_ids']: {type(batch['input_ids'])}")
                print(f"Content of batch['input_ids']: {batch['input_ids']}")

                # Convert to tensor (if needed) and move to device
                if not isinstance(batch['input_ids'], torch.Tensor):
                    input_ids = torch.tensor(batch['input_ids'], dtype=torch.long).to(device)
                else:
                    input_ids = batch['input_ids'].to(device)

                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                _, predictions = torch.max(outputs.logits, dim=1)

                all_predictions.append(predictions.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
            except Exception as e:
                print(f"Exception encountered: {e}")
                break  # Break out of the loop if we encounter an exception

        all_predictions = np.concatenate([np.array(preds) for preds in all_predictions], axis=0)
        all_labels = np.concatenate([np.array(labels) for labels in all_labels], axis=0)


        if task == "sst2":
            results[task] = {"accuracy": metrics[task].compute(predictions=all_predictions, references=all_labels)}
        elif task == "mrpc":
            results[task] = {"f1": metrics[task].compute(predictions=all_predictions, references=all_labels)["f1"]}
        elif task == "cola":
            results[task] = {"matthews_correlation": metrics[task].compute(predictions=all_predictions, references=all_labels)["matthews_correlation"]}

print(results)


Type of batch['input_ids']: <class 'list'>
Content of batch['input_ids']: [tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([  405,   879, 37984,   627,   405, 24648,   102,   368]), tensor([  128,  4825,   201,  3501,   128, 31923,  2128,   608]), tensor([   29,  3796,     7,  2156,    29,    19, 35138,    94]), tensor([   10,  7790,  1034, 14690,  2635, 12073,   822,    76]), tensor([18452, 23530,    14,  2156,   480,     8,   479,   128]), tensor([   8,    8,  295,  930,  182,   10, 1437,   29]), tensor([  747,  7764, 13887,  2156,  2156,   367,     2,  2556]), tensor([ 7920,  1437,    16, 18535,   182, 33639,     1,    19]), tensor([ 3251,     2, 10137, 34226,  2635, 16170,     1,   110]), tensor([  479,     1,     7,     8,   479, 12325,     1,  1931]), tensor([ 1437,     1, 14976,  2369,  1437,  2156,     1,    12]), tensor([    2,     1,    10,    32,     2,     5,     1, 12295]), tensor([  1,   1, 538,  70,   1, 822,   1, 479]), tensor([    1,     1,   756, 29932,     1,    16,     1, 

ValueError: ignored

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

# Load datasets
tasks = ["sst2"]
glue_data = {task: load_dataset("glue", task) for task in tasks}

# Load tokenizer and create student model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = create_half_size_roberta_base()
state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/student_model_state2.pth")
model.load_state_dict(state_dict)
model.to(device)

# Preprocess the data
def encode(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=256, return_tensors="pt")

encoded_data = {split: glue_data["sst2"][split].map(encode, batched=True) for split in ["train", "validation", "test"]}

# Collation function
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'label': torch.tensor([item['label'] for item in batch])
    }

# Prediction function
def predict(model, data_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            inputs = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(inputs, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["label"].numpy())

    return predictions, true_labels

data_loader = torch.utils.data.DataLoader(encoded_data["validation"], batch_size=32, collate_fn=collate_fn)
predictions, true_labels = predict(model, data_loader)

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.8039
F1 Score: 0.8029


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_half_size_roberta_base():
    student_config = RobertaConfig.from_pretrained("roberta-base")

    student_config.hidden_size //= 2
    student_config.num_attention_heads //= 2
    student_config.intermediate_size //= 2
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

# Load datasets
tasks = ["sst2", "cola", "mrpc"]
glue_data = {task: load_dataset("glue", task) for task in tasks}

# Load tokenizer and create student model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = create_half_size_roberta_base()
state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/student_model_state2.pth")
model.load_state_dict(state_dict)
model.to(device)

# Preprocess the data
def encode(task, examples):
    if task == "sst2":
        return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    elif task == "cola":
        return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    elif task == "mrpc":
        return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length", max_length=256, return_tensors="pt")

encoded_data = {task: {split: glue_data[task][split].map(lambda examples: encode(task, examples), batched=True) for split in ["train", "validation", "test"]} for task in tasks}

# Collation function
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in batch]),
        'label': torch.tensor([item['label'] for item in batch])
    }

# Prediction function
def predict(model, data_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            inputs = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(inputs, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["label"].numpy())

    return predictions, true_labels

for task in tasks:
    print(f"Evaluating on {task.upper()}")
    data_loader = torch.utils.data.DataLoader(encoded_data[task]["validation"], batch_size=32, collate_fn=collate_fn)
    predictions, true_labels = predict(model, data_loader)

    # Calculate accuracy, F1 score, and Matthews correlation coefficient
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    mcc = matthews_corrcoef(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Matthews correlation coefficient: {mcc:.4f}\n")


Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Evaluating on SST2
Accuracy: 0.8039
F1 Score: 0.8029
Matthews correlation coefficient: 0.6170

Evaluating on COLA
Accuracy: 0.3969
F1 Score: 0.3937
Matthews correlation coefficient: -0.0587

Evaluating on MRPC
Accuracy: 0.3995
F1 Score: 0.3879
Matthews correlation coefficient: -0.0383



In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig

def print_total_parameters_in_millions(model, model_name):
    print(f"Total Parameters for {model_name}:")
    total_params = sum(p.numel() for p in model.parameters())
    total_params_millions = total_params / 1_000_000  # Convert to millions
    print(f"{total_params_millions:.2f}M\n")  # Display 2 decimal points
    print("-" * 50 + "\n")

def print_model_config(model, model_name):
    print(f"Configuration for {model_name}:")
    print(model.config)
    print("-" * 50 + "\n")

# Initialize the teacher model (roberta-large)
teacher_model = RobertaForSequenceClassification.from_pretrained("roberta-large")

# Initialize the student model from roberta-base with custom configuration
def create_custom_student_model():
    student_config = RobertaConfig.from_pretrained("roberta-base")
    student_config.hidden_size = 512
    student_config.num_attention_heads = 8
    student_config.num_hidden_layers = 12
    student_config.intermediate_size = 1536
    student_model = RobertaForSequenceClassification(student_config)
    return student_model

student_model = create_custom_student_model()

# Initialize the actual roberta-base model
roberta_base_model = RobertaForSequenceClassification.from_pretrained("roberta-base")

print_total_parameters_in_millions(teacher_model, "Teacher Model (roberta-large)")
print_model_config(teacher_model, "Teacher Model (roberta-large)")

print_total_parameters_in_millions(student_model, "Student Model (roberta-base with custom configuration)")
print_model_config(student_model, "Student Model (roberta-base with custom configuration)")

print_total_parameters_in_millions(roberta_base_model, "Actual roberta-base Model")
print_model_config(roberta_base_model, "Actual roberta-base Model")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters for Teacher Model (roberta-large):
355.36M

--------------------------------------------------

Configuration for Teacher Model (roberta-large):
RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

--------------------------------------------------

Total Parameters for Student Model (roberta-base with custom configuration):
57.80M

----------------------------------------